In [1]:
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from itertools import chain

### NIH Reporter Search
* Fiscal Year: Active Projects
* Combine multiple categories with: OR
* NIH Spending Category: HIV/AIDS
* Project Start Date: On or After: 1/1/2019


In [ ]:
## Upload a .csv from NIH Reporter

## "data/2022_03-22 FSM Grants 2017-2022.csv"
## "data/utpractice.csv"
## "data/2022_07-18 NIH Reporter HIV AIDS.csv"
## "data/SearchResult_Export_25Jul2022_035930.csv"
reporter_data_path = "data/SearchResult_Export_25Jul2022_035930.csv"

## Read the CSV file and store into Pandas DataFrame 
reporter_data_df = pd.read_csv(reporter_data_path, encoding = "ISO-8859-1")
## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
reporter_data_df.columns =  reporter_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
reporter_data_df.head()
    

In [ ]:
## Add a column to contain link to NIH Reporter for project details
reporter_data_df['link_reporter'] = 'https://reporter.nih.gov/project-details/' + reporter_data_df['application_id'].astype(str)
reporter_data_df.head()

In [ ]:
print(reporter_data_df.columns)

In [ ]:
## Merge these columns to create lists: Funding ICs, Direct Costs, InDirect Cjosts, Total IC Costs

reporter_data_agg_df = reporter_data_df.groupby('application_id', as_index=False).agg({
                                                                    'ï»¿nih_spending_categorization':'first', 
                                                                    'project_terms':'first', 
                                                                    'project_title':'first',
                                                                    'public_health_relevance':'first', 
                                                                    'administering_ic':'first', 
                                                                    'application_id':'first',
                                                                    'award_notice_date':'first', 
                                                                    'foa':'first', 
                                                                    'project_number':'first', 
                                                                    'type':'first', 
                                                                    'activity':'first', 
                                                                    'ic':'first',
                                                                    'serial_number':'first', 
                                                                    'support_year':'first', 
                                                                    'suffix':'first',
                                                                    'program_official_information':'first', 
                                                                    'project_start_date':'first',
                                                                    'project_end_date':'first', 
                                                                    'study_section':'first', 
                                                                    'subproject_number':'first',
                                                                    'contact_pi_person_id':'first', 
                                                                    'contact_pi_/_project_leader':'first',
                                                                    'other_pi_or_project_leaders':'first', 
                                                                    'congressional_district':'first', 
                                                                    'department':'first',
                                                                    'primary_duns':'first', 
                                                                    'primary_uei':'first', 
                                                                    'duns_number':'first', 
                                                                    'uei':'first', 
                                                                    'fips':'first', 
                                                                    'latitude':'first',
                                                                    'longitude':'first', 
                                                                    'organization_id_ipf':'first', 
                                                                    'organization_name':'first',
                                                                    'organization_city':'first', 
                                                                    'organization_state':'first', 
                                                                    'organization_type':'first',
                                                                    'organization_zip':'first', 
                                                                    'organization_country':'first', 
                                                                    'arra_indicator':'first',
                                                                    'budget_start_date':'first', 
                                                                    'budget_end_date':'first', 
                                                                    'cfda_code':'first',
                                                                    'funding_mechanism':'first', 
                                                                    'fiscal_year':'first', 
                                                                    'total_cost':'first',
                                                                    'total_cost_sub_projects':'first', 
                                                                    'funding_ics': list, 
                                                                    'direct_cost_ic': list,
                                                                    'indirect_cost_ic': list, 
                                                                    'nih_covid-19_response' :'first', 
                                                                    #'project_abstract':'first',
                                                                    'total_cost_ic':list, 
                                                                    'link_reporter': 'first'
    
                                                                      })
reporter_data_agg_df.head()


In [ ]:
## Filter to include ONLY projects with "organizational country" in United States 

countries = ['UNITED STATES'] 
    
# selecting rows based on condition 
reporter_data_query_df = reporter_data_agg_df.loc[reporter_data_agg_df['organization_country'].isin(countries)]
    
reporter_data_query_df.head()


In [ ]:
## Add column for Region
## https://stackoverflow.com/questions/41189392/new-column-in-pandas-dataframe-based-on-existing-column-values

regions = {
    'WA': 'West', 'OR': 'West', 'CA': 'West', 'NV': 'West',
    'ID': 'West', 'MT': 'West', 'WY': 'West', 'UT': 'West',
    'CO': 'West', 'AK': 'West', 'HI': 'West', 'ME': 'Northeast',
    'VT': 'Northeast', 'NY': 'Northeast', 'NH': 'Northeast',
    'MA': 'Northeast', 'RI': 'Northeast', 'CT': 'Northeast',
    'NJ': 'Northeast', 'PA': 'Northeast', 'ND': 'Midwest',
    'SD': 'Midwest', 'NE': 'Midwest', 'KS': 'Midwest',
    'MN': 'Midwest', 'IA': 'Midwest', 'MO': 'Midwest', 'WI': 'Midwest',
    'IL': 'Midwest', 'MI': 'Midwest', 'IN': 'Midwest', 'OH': 'Midwest',
    'WV': 'South', 'DC': 'South', 'MD': 'South',
    'VA': 'South', 'KY': 'South', 'TN': 'South', 'NC': 'South',
    'MS': 'South', 'AR': 'South', 'LA': 'South', 'AL': 'South',
    'GA': 'South', 'SC': 'South', 'FL': 'South', 'DE': 'South',
    'AZ': 'Southwest', 'NM': 'Southwest', 'OK': 'Southwest',
    'TX': 'Southwest', 'AB': 'Canada', 'BC': 'Canada', 'MB': 'Canada', 
    'NB': 'Canada', 'NL': 'Canada', 'NT': 'Canada', 'NS': 'Canada', 
    'NU': 'Canada', 'ON': 'Canada', 'PE': 'Canada', 'QC': 'Canada', 
    'SK': 'Canada', 'YT': 'Canada', 'PQ':'Cananda' ,'PR': 'Puerto Rico', 'None': 'None' }


def get_region(state):
#     print(state)
#     print(regions[state])
    return regions[state]

reporter_data_query_df['Region'] = reporter_data_query_df['organization_state'].fillna("None").apply(get_region)
reporter_data_query_df.head()

## Other helpful code
## https://gist.github.com/sfirrin/fd01d87f022d80e98c37a045c14109fe

# states_to_regions = {
#     'Washington': 'West', 'Oregon': 'West', 'California': 'West', 'Nevada': 'West',
#     'Idaho': 'West', 'Montana': 'West', 'Wyoming': 'West', 'Utah': 'West',
#     'Colorado': 'West', 'Alaska': 'West', 'Hawaii': 'West', 'Maine': 'Northeast',
#     'Vermont': 'Northeast', 'New York': 'Northeast', 'New Hampshire': 'Northeast',
#     'Massachusetts': 'Northeast', 'Rhode Island': 'Northeast', 'Connecticut': 'Northeast',
#     'New Jersey': 'Northeast', 'Pennsylvania': 'Northeast', 'North Dakota': 'Midwest',
#     'South Dakota': 'Midwest', 'Nebraska': 'Midwest', 'Kansas': 'Midwest',
#     'Minnesota': 'Midwest', 'Iowa': 'Midwest', 'Missouri': 'Midwest', 'Wisconsin': 'Midwest',
#     'Illinois': 'Midwest', 'Michigan': 'Midwest', 'Indiana': 'Midwest', 'Ohio': 'Midwest',
#     'West Virginia': 'South', 'District of Columbia': 'South', 'Maryland': 'South',
#     'Virginia': 'South', 'Kentucky': 'South', 'Tennessee': 'South', 'North Carolina': 'South',
#     'Mississippi': 'South', 'Arkansas': 'South', 'Louisiana': 'South', 'Alabama': 'South',
#     'Georgia': 'South', 'South Carolina': 'South', 'Florida': 'South', 'Delaware': 'South',
#     'Arizona': 'Southwest', 'New Mexico': 'Southwest', 'Oklahoma': 'Southwest',
#     'Texas': 'Southwest'}

# regions_to_states = {
#     'South': ['West Virginia', 'District of Columbia', 'Maryland', 'Virginia',
#               'Kentucky', 'Tennessee', 'North Carolina', 'Mississippi',
#               'Arkansas', 'Louisiana', 'Alabama', 'Georgia', 'South Carolina',
#               'Florida', 'Delaware'],
#     'Southwest': ['Arizona', 'New Mexico', 'Oklahoma', 'Texas'],
#     'West': ['Washington', 'Oregon', 'California', 'Nevada', 'Idaho', 'Montana',
#              'Wyoming', 'Utah', 'Colorado', 'Alaska', 'Hawaii'],
#     'Midwest': ['North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota',
#                 'Iowa', 'Missouri', 'Wisconsin', 'Illinois', 'Michigan', 'Indiana',
#                 'Ohio'],
#     'Northeast': ['Maine', 'Vermont', 'New York', 'New Hampshire', 'Massachusetts',
#                   'Rhode Island', 'Connecticut', 'New Jersey', 'Pennsylvania']
# }


In [ ]:
## Export file to excel, without the Pandas index, but with the header

reporter_data_query_df.to_excel("output/aggregated_data.xlsx", index=False, header=True)

In [ ]:
reporter_data_combined_org_df = reporter_data_query_df.groupby('organization_name', as_index=False).agg({
                                                                    'latitude':'first',
                                                                    'longitude':'first', 
                                                                    'organization_id_ipf':'first', 
                                                                    'organization_name':'first',
                                                                    'organization_city':'first', 
                                                                    'organization_state':'first', 
                                                                    'organization_type':'first',
                                                                    'organization_zip':'first', 
                                                                    'organization_country':'first', 
                                                                    'arra_indicator':'first',
                                                                    'budget_start_date':'first', 
                                                                    'budget_end_date':'first', 
                                                                    'project_start_date' : 'first',
                                                                    'project_end_date' : 'first',
                                                                    'fiscal_year':'first', 
                                                                    'total_cost':'sum',
                                                                    'Region':'first',
                                                                    'link_reporter': 'first'
    
                                                                      })
reporter_data_combined_org_df.head()

In [ ]:
## Matplotlib scatter plot
plt.scatter(x=reporter_data_agg_df['longitude'], y=reporter_data_agg_df['latitude'])
plt.show()

In [ ]:
import plotly.express as px
reporter_data_combined_org_df['total_cost'] = reporter_data_combined_org_df['total_cost'].fillna(0)
reporter_data_combined_org_df['project_end_date'] = reporter_data_combined_org_df['project_end_date'].fillna(0)
df = reporter_data_combined_org_df

fig = px.scatter_geo(df, 
                     lon='longitude', 
                     lat = 'latitude', 
                     hover_name= 'organization_name', 
                     size = 'total_cost',
                     scope="north america",
                     #locationmode = 'USA-states',
                     #animation_frame='project_end_date'
                     color= 'Region'
                    )
# fig = go.Figure(data=go.Scattergeo(
#         lon = df['long'],
#         lat = df['lat'],
#         text = df['text'],
#         mode = 'markers',
#         marker_color = df['cnt'],
#         ))

fig.update_layout(
        title = 'NIH Funding by Institution',
        geo_scope='usa',
    )
fig.show()


In [ ]:
## Upload a .csv from NIH Reporter

funding_institutional_data_path = "data/funding institutional.csv"

## Read the CSV file and store into Pandas DataFrame 
funding_institutional_data_df = pd.read_csv(funding_institutional_data_path 
                                            ,encoding = "ISO-8859-1")
                                           

## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']

#Change the column names to lower case with underscore for spaces
funding_institutional_data_df.columns =  funding_institutional_data_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
funding_institutional_data_df.head()
    

In [ ]:
## Prepare Aggregated Data Data Frame (which is NIH Reporter data) for merging with Funding Institutional 

## Create a new column and add Project Number while removing the data after the hyphen 
## https://stackoverflow.com/questions/69022832/pandas-how-to-remove-only-the-last-hyphen-at-the-end-of-a-column

reporter_data_agg_df["project_number_matching"] = reporter_data_agg_df['project_number'].str.replace('-[^-]*$', '')


## Then remove the first number of the project number

reporter_data_agg_df["project_number_matching"] = reporter_data_agg_df["project_number_matching"].str[1:]

reporter_data_agg_df.head()

In [ ]:
## Merge "amount" from Funding Institutional with NIH Reporter using funding_number and project_number_matching
 
merged_df = pd.merge(reporter_data_agg_df, funding_institutional_data_df, how='left', left_on =['project_number_matching'], right_on=['funding_number'])

merged_df.head()

In [ ]:
## Export file to excel, without the Pandas index, but with the header

merged_df.to_excel("output/merged_data.xlsx", index=False, header=True)

### NIH Reporter Search
* Fiscal Year: 2022, 2021, 2020, 2019
* Combine multiple categories with: OR
* NIH Spending Category: HIV/AIDS

In [7]:
## Upload a .csv from NIH Reporter

reporter_data_2_path = "data/2022_07-29 NIH Reporter Search.csv"

## Read the CSV file and store into Pandas DataFrame 
reporter_data_2_df = pd.read_csv(reporter_data_2_path
                                 ,encoding = "ISO-8859-1"
                                 ,dtype={
                                    'NIH Spending Categorization':str, 
                                    'Project Terms':str, 
                                    'project_title':str,
                                    'public_health_relevance':str,
                                    'administering_ic':str,
                                    'application_id':str,
                                    'award_notice_date':str,
                                    'foa':str, 
                                    'project_number':str,
                                    'type':str,
                                    'activity':str,
                                    'ic':str,
                                    'Serial Number':str,
                                    'support_year':str,
                                    'suffix':str,
                                    'program_official_information':str,
                                    'project_start_date':str,
                                    'project_end_date':str,
                                    'study_section':str,
                                    'subproject_number':str,
                                    'contact_pi_person_id':str,
                                    'contact_pi_/_project_leader':str,
                                    'other_pi_or_project_leaders':str,
                                    'congressional_district':str, 
                                    'department':str,
                                    'primary_duns':str,
                                    'primary_uei':str,
                                    'duns_number':str,
                                    'uei':str,
                                    'fips':str,
                                    'latitude':float,
                                    'longitude':float,
                                    'organization_id_ipf':str,
                                    'organization_name':str,
                                    'organization_city':str,
                                    'organization_state':str,
                                    'organization_type':str,
                                    'organization_zip':str,
                                    'organization_country':str,
                                    'arra_indicator':str,
                                    'budget_start_date':str,
                                    'budget_end_date':str,
                                    'cfda_code':str,
                                    'funding_mechanism':str,
                                    'fiscal_year':str,
                                    'total_cost':float,
                                    'total_cost_sub_projects':float,
                                    'funding_ics': str, 
                                    'direct_cost_ic':str,
                                    'indirect_cost_ic': str, 
                                    'nih_covid-19_response' :str,
                                    #'project_abstract':'first',
                                    'total_cost_ic': float})                                                 
                                                

## encoding = "ISO-8859-1", na_values=['NULL', '<NA>']
## converters = {'serial_number': object}

#Change the column names to lower case with underscore for spaces
reporter_data_2_df.columns =  reporter_data_2_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
reporter_data_2_df.head()
    

C:\Users\keg827\AppData\Local\Temp\ipykernel_28284\1892256442.py:68: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reporter_data_2_df.columns =  reporter_data_2_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
C:\Users\keg827\AppData\Local\Temp\ipykernel_28284\1892256442.py:68: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  reporter_data_2_df.columns =  reporter_data_2_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")


,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,cfda_code,funding_mechanism,fiscal_year,total_cost,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic
0,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,9961667,7/27/2020,PA-18-363,5K01MH118943-02,5.0,...,242.0,Other Research-Related,2020,183667.0,NaN,NIMH,170581.0,13086.0,NaN,183667.0
1,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,9779087,6/20/2019,PA-18-363,1K01MH118943-01A1,1.0,...,242.0,Other Research-Related,2019,183408.0,NaN,NIMH,170600.0,12808.0,NaN,183408.0
2,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,10223987,6/28/2021,PA-18-363,5K01MH118943-03,5.0,...,242.0,Other Research-Related,2021,183885.0,NaN,NIMH,170523.0,13362.0,NaN,183885.0
3,Cancer; Clinical Research; Clinical Trials and...,Area;Cancer Trials Support Unit;Clinical Trial...,Cancer Trials Support Unit,NaN,NCI,10005864,NaN,NaN,261201300028I-P00011-26100001-1,NaN,...,NaN,R and D Contracts,2019,16340000.0,NaN,NCI,NaN,NaN,NaN,16340000.0
4,Cancer; Clinical Research; Clinical Trials and...,Area;Cancer Trials Support Unit;Clinical Trial...,Cancer Trials Support Unit,NaN,NCI,10188329,NaN,NaN,261201300028I-P00013-26100001-1,NaN,...,NaN,R and D Contracts,2020,625000.0,NaN,NCI,NaN,NaN,NaN,625000.0


In [8]:
## Add a column to contain link to NIH Reporter for project details
reporter_data_2_df['link_reporter'] = 'https://reporter.nih.gov/project-details/' + reporter_data_2_df['application_id'].astype(str)
reporter_data_2_df.head()

,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,funding_mechanism,fiscal_year,total_cost,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic,link_reporter
0,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,9961667,7/27/2020,PA-18-363,5K01MH118943-02,5.0,...,Other Research-Related,2020,183667.0,NaN,NIMH,170581.0,13086.0,NaN,183667.0,https://reporter.nih.gov/project-details/9961667
1,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,9779087,6/20/2019,PA-18-363,1K01MH118943-01A1,1.0,...,Other Research-Related,2019,183408.0,NaN,NIMH,170600.0,12808.0,NaN,183408.0,https://reporter.nih.gov/project-details/9779087
2,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,10223987,6/28/2021,PA-18-363,5K01MH118943-03,5.0,...,Other Research-Related,2021,183885.0,NaN,NIMH,170523.0,13362.0,NaN,183885.0,https://reporter.nih.gov/project-details/10223987
3,Cancer; Clinical Research; Clinical Trials and...,Area;Cancer Trials Support Unit;Clinical Trial...,Cancer Trials Support Unit,NaN,NCI,10005864,NaN,NaN,261201300028I-P00011-26100001-1,NaN,...,R and D Contracts,2019,16340000.0,NaN,NCI,NaN,NaN,NaN,16340000.0,https://reporter.nih.gov/project-details/10005864
4,Cancer; Clinical Research; Clinical Trials and...,Area;Cancer Trials Support Unit;Clinical Trial...,Cancer Trials Support Unit,NaN,NCI,10188329,NaN,NaN,261201300028I-P00013-26100001-1,NaN,...,R and D Contracts,2020,625000.0,NaN,NCI,NaN,NaN,NaN,625000.0,https://reporter.nih.gov/project-details/10188329


In [9]:
print(reporter_data_2_df.columns)

Index(['nih_spending_categorization', 'project_terms', 'project_title',
       'public_health_relevance', 'administering_ic', 'application_id',
       'award_notice_date', 'foa', 'project_number', 'type', 'activity', 'ic',
       'serial_number', 'support_year', 'suffix',
       'program_official_information', 'project_start_date',
       'project_end_date', 'study_section', 'subproject_number',
       'contact_pi_person_id', 'contact_pi_/_project_leader',
       'other_pi_or_project_leaders', 'congressional_district', 'department',
       'primary_duns', 'primary_uei', 'duns_number', 'uei', 'fips', 'latitude',
       'longitude', 'organization_id_ipf', 'organization_name',
       'organization_city', 'organization_state', 'organization_type',
       'organization_zip', 'organization_country', 'arra_indicator',
       'budget_start_date', 'budget_end_date', 'cfda_code',
       'funding_mechanism', 'fiscal_year', 'total_cost',
       'total_cost_sub_projects', 'funding_ics', 'direct_cos

In [10]:
test = reporter_data_2_df["serial_number"]
test.dtype

dtype('O')

In [11]:
## Concatenate columns for IC and Serial Number to create a unique ID for each year

reporter_data_2_df["unique_id"] = reporter_data_2_df['ic'] + reporter_data_2_df['serial_number'].astype(str)

reporter_data_2_df.head()

,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,fiscal_year,total_cost,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic,link_reporter,unique_id
0,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,9961667,7/27/2020,PA-18-363,5K01MH118943-02,5.0,...,2020,183667.0,NaN,NIMH,170581.0,13086.0,NaN,183667.0,https://reporter.nih.gov/project-details/9961667,MH118943
1,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,9779087,6/20/2019,PA-18-363,1K01MH118943-01A1,1.0,...,2019,183408.0,NaN,NIMH,170600.0,12808.0,NaN,183408.0,https://reporter.nih.gov/project-details/9779087,MH118943
2,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,10223987,6/28/2021,PA-18-363,5K01MH118943-03,5.0,...,2021,183885.0,NaN,NIMH,170523.0,13362.0,NaN,183885.0,https://reporter.nih.gov/project-details/10223987,MH118943
3,Cancer; Clinical Research; Clinical Trials and...,Area;Cancer Trials Support Unit;Clinical Trial...,Cancer Trials Support Unit,NaN,NCI,10005864,NaN,NaN,261201300028I-P00011-26100001-1,NaN,...,2019,16340000.0,NaN,NCI,NaN,NaN,NaN,16340000.0,https://reporter.nih.gov/project-details/10005864,NaN
4,Cancer; Clinical Research; Clinical Trials and...,Area;Cancer Trials Support Unit;Clinical Trial...,Cancer Trials Support Unit,NaN,NCI,10188329,NaN,NaN,261201300028I-P00013-26100001-1,NaN,...,2020,625000.0,NaN,NCI,NaN,NaN,NaN,625000.0,https://reporter.nih.gov/project-details/10188329,NaN


In [12]:
## Filter to include ONLY projects with "organizational country" in United States 

countries = ['UNITED STATES'] 
    
# selecting rows based on condition 
reporter_data_2_df = reporter_data_2_df.loc[reporter_data_2_df['organization_country'].isin(countries)]
    
reporter_data_2_df.head()

,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,fiscal_year,total_cost,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic,link_reporter,unique_id
0,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,9961667,7/27/2020,PA-18-363,5K01MH118943-02,5.0,...,2020,183667.0,NaN,NIMH,170581.0,13086.0,NaN,183667.0,https://reporter.nih.gov/project-details/9961667,MH118943
1,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,9779087,6/20/2019,PA-18-363,1K01MH118943-01A1,1.0,...,2019,183408.0,NaN,NIMH,170600.0,12808.0,NaN,183408.0,https://reporter.nih.gov/project-details/9779087,MH118943
2,Behavioral and Social Science; Clinical Resear...,AIDS prevention;Acquired Immunodeficiency Synd...,A Multicomponent Intervention to Increase HIV...,PROJECT NARRATIVE\n The proposed study offers ...,NIMH,10223987,6/28/2021,PA-18-363,5K01MH118943-03,5.0,...,2021,183885.0,NaN,NIMH,170523.0,13362.0,NaN,183885.0,https://reporter.nih.gov/project-details/10223987,MH118943
3,Cancer; Clinical Research; Clinical Trials and...,Area;Cancer Trials Support Unit;Clinical Trial...,Cancer Trials Support Unit,NaN,NCI,10005864,NaN,NaN,261201300028I-P00011-26100001-1,NaN,...,2019,16340000.0,NaN,NCI,NaN,NaN,NaN,16340000.0,https://reporter.nih.gov/project-details/10005864,NaN
4,Cancer; Clinical Research; Clinical Trials and...,Area;Cancer Trials Support Unit;Clinical Trial...,Cancer Trials Support Unit,NaN,NCI,10188329,NaN,NaN,261201300028I-P00013-26100001-1,NaN,...,2020,625000.0,NaN,NCI,NaN,NaN,NaN,625000.0,https://reporter.nih.gov/project-details/10188329,NaN


In [22]:
## Create a list of SubAwards for each Grant Award, so there are no duplicate columns for Unique IDs EXCEPT for by year. 
## Group projects by their unique id, and aggregate these columns to create lists: Funding ICs, Direct Costs, InDirect Cjosts, Total IC Costs

reporter_data_2_agg_df = reporter_data_2_df.groupby(['unique_id','fiscal_year'], as_index=False).agg({
                                                                    'nih_spending_categorization':'first', 
                                                                    'project_terms':'first', 
                                                                    'project_title':'first',
                                                                    'public_health_relevance':'first', 
                                                                    'administering_ic':'first', 
                                                                    'application_id':'first',
                                                                    'award_notice_date':'first', 
                                                                    'foa':'first', 
                                                                    'project_number':'first', 
                                                                    'type':'first', 
                                                                    'activity':'first', 
                                                                    'ic':'first',
                                                                    'serial_number':'first', 
                                                                    'support_year':'first', 
                                                                    'suffix':'first',
                                                                    'program_official_information':'first', 
                                                                    'project_start_date':'first',
                                                                    'project_end_date':'first', 
                                                                    'study_section':'first', 
                                                                    'subproject_number':'first',
                                                                    'contact_pi_person_id':'first', 
                                                                    'contact_pi_/_project_leader':'first',
                                                                    'other_pi_or_project_leaders':'first', 
                                                                    'congressional_district':'first', 
                                                                    'department':'first',
                                                                    'primary_duns':'first', 
                                                                    'primary_uei':'first', 
                                                                    'duns_number':'first', 
                                                                    'uei':'first', 
                                                                    'fips':'first', 
                                                                    'latitude':'first',
                                                                    'longitude':'first', 
                                                                    'organization_id_ipf':'first', 
                                                                    'organization_name':'first',
                                                                    'organization_city':'first', 
                                                                    'organization_state':'first', 
                                                                    'organization_type':'first',
                                                                    'organization_zip':'first', 
                                                                    'organization_country':'first', 
                                                                    'arra_indicator':'first',
                                                                    'budget_start_date':'first', 
                                                                    'budget_end_date':'first', 
                                                                    'cfda_code':'first',
                                                                    'funding_mechanism':'first', 
                                                                    'fiscal_year':'first', 
                                                                    'total_cost':'first',
                                                                    'total_cost_sub_projects':'first', 
                                                                    'funding_ics': list, 
                                                                    'direct_cost_ic': 'sum',
                                                                    'indirect_cost_ic': 'sum', 
                                                                    'nih_covid-19_response' :'first', 
                                                                    #'project_abstract':'first',
                                                                    'total_cost_ic':list, 
                                                                    'link_reporter': 'first',
                                                                    'unique_id': 'first'
    
                                                                      })
reporter_data_2_agg_df.head()

,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,fiscal_year,total_cost,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic,link_reporter,unique_id
0,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,9717066,8/30/2019,PA-18-403,2T32AA007577-21,2.0,...,2019,316306.0,NaN,[NIAAA],483697.0,32609.0,None,[316306.0],https://reporter.nih.gov/project-details/9717066,AA007577
1,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,10005102,8/19/2020,PA-18-403,5T32AA007577-22,5.0,...,2020,432942.0,NaN,[NIAAA],501017.0,33995.0,None,[432942.0],https://reporter.nih.gov/project-details/10005102,AA007577
2,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,10241974,8/31/2021,PA-18-403,5T32AA007577-23,5.0,...,2021,446399.0,NaN,[NIAAA],507857.0,34542.0,None,[446399.0],https://reporter.nih.gov/project-details/10241974,AA007577
3,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",AIDS/HIV problem;Alcohol abuse;Alcohol consump...,Administrative Core,None,NIAAA,9599422,11/26/2018,RFA-AA-14-003,5P60AA009803-26,5.0,...,2019,NaN,131701.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1771755.0,639931.0,None,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",https://reporter.nih.gov/project-details/9599422,AA009803
4,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",AIDS/HIV problem;Address;Alcohol abuse;Alcohol...,Administrative Core,None,NIAAA,9917261,12/20/2019,RFA-AA-19-003,2P60AA009803-27,2.0,...,2020,NaN,230037.0,"[nan, nan, nan, nan, nan, nan, nan]",1066553.0,450706.0,None,"[nan, nan, nan, nan, nan, nan, nan]",https://reporter.nih.gov/project-details/9917261,AA009803


In [23]:
## Export file to excel, without the Pandas index, but with the header

reporter_data_2_agg_df.to_excel("output/reporter_data_2_agg_df.xlsx", index=False, header=True)

In [24]:
## Add column for Region
## https://stackoverflow.com/questions/41189392/new-column-in-pandas-dataframe-based-on-existing-column-values

regions = {
    'WA': 'West', 'OR': 'West', 'CA': 'West', 'NV': 'West',
    'ID': 'West', 'MT': 'West', 'WY': 'West', 'UT': 'West',
    'CO': 'West', 'AK': 'West', 'HI': 'West', 'ME': 'Northeast',
    'VT': 'Northeast', 'NY': 'Northeast', 'NH': 'Northeast',
    'MA': 'Northeast', 'RI': 'Northeast', 'CT': 'Northeast',
    'NJ': 'Northeast', 'PA': 'Northeast', 'ND': 'Midwest',
    'SD': 'Midwest', 'NE': 'Midwest', 'KS': 'Midwest',
    'MN': 'Midwest', 'IA': 'Midwest', 'MO': 'Midwest', 'WI': 'Midwest',
    'IL': 'Midwest', 'MI': 'Midwest', 'IN': 'Midwest', 'OH': 'Midwest',
    'WV': 'South', 'DC': 'South', 'MD': 'South',
    'VA': 'South', 'KY': 'South', 'TN': 'South', 'NC': 'South',
    'MS': 'South', 'AR': 'South', 'LA': 'South', 'AL': 'South',
    'GA': 'South', 'SC': 'South', 'FL': 'South', 'DE': 'South',
    'AZ': 'Southwest', 'NM': 'Southwest', 'OK': 'Southwest',
    'TX': 'Southwest', 'AB': 'Canada', 'BC': 'Canada', 'MB': 'Canada', 
    'NB': 'Canada', 'NL': 'Canada', 'NT': 'Canada', 'NS': 'Canada', 
    'NU': 'Canada', 'ON': 'Canada', 'PE': 'Canada', 'QC': 'Canada', 
    'SK': 'Canada', 'YT': 'Canada', 'PQ':'Cananda' ,'PR': 'Puerto Rico', 'None': 'None' }


def get_region(state):
#     print(state)
#     print(regions[state])
    return regions[state]

reporter_data_2_agg_df['Region'] = reporter_data_2_agg_df['organization_state'].fillna("None").apply(get_region)
reporter_data_2_agg_df.head()


,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,total_cost,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic,link_reporter,unique_id,Region
0,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,9717066,8/30/2019,PA-18-403,2T32AA007577-21,2.0,...,316306.0,NaN,[NIAAA],483697.0,32609.0,None,[316306.0],https://reporter.nih.gov/project-details/9717066,AA007577,South
1,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,10005102,8/19/2020,PA-18-403,5T32AA007577-22,5.0,...,432942.0,NaN,[NIAAA],501017.0,33995.0,None,[432942.0],https://reporter.nih.gov/project-details/10005102,AA007577,South
2,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,10241974,8/31/2021,PA-18-403,5T32AA007577-23,5.0,...,446399.0,NaN,[NIAAA],507857.0,34542.0,None,[446399.0],https://reporter.nih.gov/project-details/10241974,AA007577,South
3,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",AIDS/HIV problem;Alcohol abuse;Alcohol consump...,Administrative Core,None,NIAAA,9599422,11/26/2018,RFA-AA-14-003,5P60AA009803-26,5.0,...,NaN,131701.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1771755.0,639931.0,None,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",https://reporter.nih.gov/project-details/9599422,AA009803,South
4,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",AIDS/HIV problem;Address;Alcohol abuse;Alcohol...,Administrative Core,None,NIAAA,9917261,12/20/2019,RFA-AA-19-003,2P60AA009803-27,2.0,...,NaN,230037.0,"[nan, nan, nan, nan, nan, nan, nan]",1066553.0,450706.0,None,"[nan, nan, nan, nan, nan, nan, nan]",https://reporter.nih.gov/project-details/9917261,AA009803,South


In [33]:
## Create new column for "total cost" and if there's no total_cost use "total_cost_sub_projects"

reporter_data_2_agg_df['total_cost_main_or_sub_projects'] = reporter_data_2_agg_df['total_cost'].add(reporter_data_2_agg_df['total_cost_sub_projects'], fill_value=0)
reporter_data_2_agg_df.head()

## Resource
#https://stackoverflow.com/questions/53463101/pandas-summing-two-columns-with-nan

,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic,link_reporter,unique_id,Region,total_cost_main_or_sub_projects
0,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,9717066,8/30/2019,PA-18-403,2T32AA007577-21,2.0,...,NaN,[NIAAA],483697.0,32609.0,None,[316306.0],https://reporter.nih.gov/project-details/9717066,AA007577,South,316306.0
1,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,10005102,8/19/2020,PA-18-403,5T32AA007577-22,5.0,...,NaN,[NIAAA],501017.0,33995.0,None,[432942.0],https://reporter.nih.gov/project-details/10005102,AA007577,South,432942.0
2,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",Training Programs;alcohol research,Biomedical Alcohol Research Training Program,This training grant addresses the health needs...,NIAAA,10241974,8/31/2021,PA-18-403,5T32AA007577-23,5.0,...,NaN,[NIAAA],507857.0,34542.0,None,[446399.0],https://reporter.nih.gov/project-details/10241974,AA007577,South,446399.0
3,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",AIDS/HIV problem;Alcohol abuse;Alcohol consump...,Administrative Core,None,NIAAA,9599422,11/26/2018,RFA-AA-14-003,5P60AA009803-26,5.0,...,131701.0,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1771755.0,639931.0,None,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",https://reporter.nih.gov/project-details/9599422,AA009803,South,131701.0
4,"Alcoholism, Alcohol Use and Health; HIV/AIDS; ...",AIDS/HIV problem;Address;Alcohol abuse;Alcohol...,Administrative Core,None,NIAAA,9917261,12/20/2019,RFA-AA-19-003,2P60AA009803-27,2.0,...,230037.0,"[nan, nan, nan, nan, nan, nan, nan]",1066553.0,450706.0,None,"[nan, nan, nan, nan, nan, nan, nan]",https://reporter.nih.gov/project-details/9917261,AA009803,South,230037.0


In [36]:
## Group the university's projects by YEAR using the university's name. 
## Aggregate these columns to create lists: Funding ICs, Direct Costs, InDirect Cjosts, Total IC Costs

reporter_data_2_univ_df = reporter_data_2_agg_df.groupby(['organization_name', 'fiscal_year'], as_index=False).agg({
                                                                    'nih_spending_categorization': list, 
                                                                    'project_terms':list, 
                                                                    'project_title':list,
                                                                    'public_health_relevance':list, 
                                                                    'administering_ic':list, 
                                                                    'application_id':list,
                                                                    'award_notice_date':list, 
                                                                    'foa':list, 
                                                                    'project_number':list, 
                                                                    'type':list, 
                                                                    'activity':list, 
                                                                    'ic':list,
                                                                    'serial_number':list, 
                                                                    'support_year':list, 
                                                                    'suffix':list,
                                                                    'program_official_information':list, 
                                                                    'project_start_date':list,
                                                                    'project_end_date':list, 
                                                                    'study_section':list, 
                                                                    'subproject_number':list,
                                                                    'contact_pi_person_id':list, 
                                                                    'contact_pi_/_project_leader':list,
                                                                    'other_pi_or_project_leaders':list, 
                                                                    'congressional_district':list, 
                                                                    'department':list,
                                                                    'primary_duns':list, 
                                                                    'primary_uei':list, 
                                                                    'duns_number':list, 
                                                                    'uei':list, 
                                                                    'fips':list, 
                                                                    'latitude':'first',
                                                                    'longitude':'first', 
                                                                    'organization_id_ipf':'first', 
                                                                    'organization_name':'first',
                                                                    'organization_city':'first', 
                                                                    'organization_state':'first', 
                                                                    'organization_type':'first',
                                                                    'organization_zip':'first', 
                                                                    'organization_country':'first', 
                                                                    'arra_indicator':list,
                                                                    'budget_start_date':list, 
                                                                    'budget_end_date':list, 
                                                                    'cfda_code':list,
                                                                    'funding_mechanism':list, 
                                                                    'fiscal_year':'first', 
                                                                    'total_cost':'sum',
                                                                    'total_cost_sub_projects':'sum', 
                                                                    'funding_ics': list, 
                                                                    'direct_cost_ic': 'sum',
                                                                    'indirect_cost_ic': 'sum', 
                                                                    'nih_covid-19_response' :'first', 
                                                                    #'project_abstract':'first',
                                                                    'total_cost_ic':list, 
                                                                    'link_reporter': list,
                                                                    'unique_id': list,
                                                                    'Region': 'first',
                                                                    'total_cost_main_or_sub_projects':'sum'
                                                                      })
reporter_data_2_univ_df.head()

,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic,link_reporter,unique_id,Region,total_cost_main_or_sub_projects
0,"[HIV/AIDS; Infectious Diseases, Biotechnology;...",[AIDS/HIV problem;Address;Affect;Affinity;Anti...,[Diverse Functions of HIV-1 Capsid During Post...,[Project Narrative\nThe fight against HIV/AIDS...,"[NIAID, NIAID, NIAID, NIAID, NIAID, NIAID, NIMH]","[9747183, 9733093, 9736217, 9733008, 9778510, ...","[7/17/2019, 7/26/2019, 7/2/2019, 6/24/2019, 4/...","[PA-16-160, PA-13-302, PAR-16-115, PAR-14-247,...","[5R01AI100720-08, 5R01AI122953-04, 5R01AI12980...","[5.0, 5.0, 5.0, 5.0, 1.0, 1.0, 1.0]",...,0.0,"[[NIAID], [NIAID], [NIAID], [NIAID], [NIAID], ...",1391036.0,1042378.0,None,"[[76100.0], [133024.0], [443662.0], [354652.0]...",[https://reporter.nih.gov/project-details/9747...,"[AI100720, AI122953, AI129802, AI134328, AI145...",Northeast,1742747.0
1,[Clinical Research; HIV/AIDS; Health Dispariti...,[AIDS/HIV problem;Acquired Immunodeficiency Sy...,[A Novel Test to Measure Latent Virus in HIV-1...,[Project Narrative\n For HIV-1 infected indivi...,[NIAID],[9618125],[12/31/2018],[PA-16-302],[5R44AI124996-03],[5.0],...,0.0,[[NIAID]],0.0,0.0,None,[[1000000.0]],[https://reporter.nih.gov/project-details/9618...,[AI124996],South,1000000.0
2,[Clinical Research; Clinical Trials and Suppor...,[Acquired Immunodeficiency Syndrome;Address;Ad...,[Centralized Resource to Accurately Quantify L...,[PROJECT NARRATIVE\n For HIV-1 infected indivi...,"[NIAID, NIAID]","[9926700, 9907751]","[4/13/2020, 12/12/2019]","[PAR-16-412, PA-18-574]","[1U24AI143502-01A1, 1R43AI149816-01]","[1.0, 1.0]",...,0.0,"[[NIAID], [NIAID]]",499995.0,249998.0,None,"[[749993.0], [300000.0]]",[https://reporter.nih.gov/project-details/9926...,"[AI143502, AI149816]",South,1049993.0
3,[Clinical Research; Clinical Trials and Suppor...,[Acquired Immunodeficiency Syndrome;Address;Ad...,[Centralized Resource to Accurately Quantify L...,[PROJECT NARRATIVE\n For HIV-1 infected indivi...,"[NIAID, NIAID, NIAID]","[10144381, 10324540, 10324486]","[3/26/2021, 7/23/2021, 7/6/2021]","[PAR-16-412, PA-20-260, PA-20-260]","[5U24AI143502-02, 1R43AI165126-01, 1R43AI16512...","[5.0, 1.0, 1.0]",...,0.0,"[[NIAID], [NIAID], [NIAID]]",706717.0,251102.0,None,"[[749588.0], [300000.0], [300000.0]]",[https://reporter.nih.gov/project-details/1014...,"[AI143502, AI165126, AI165128]",South,1349588.0
4,[Genetics; HIV/AIDS; Infectious Diseases],[Amino Acid Transporter;Animals;Antioxidants;A...,[Understanding Nrf2-Mediated Restriction of HI...,[Project Narrative\nThe proposed research is r...,[NIAID],[9697769],[4/11/2019],[PA-16-161],[5R21AI140993-02],[5.0],...,0.0,[[NIAID]],133923.0,68282.0,None,[[202205.0]],[https://reporter.nih.gov/project-details/9697...,[AI140993],Northeast,202205.0


In [37]:
## Export file to excel, without the Pandas index, but with the header

reporter_data_2_univ_df.to_excel("output/reporter_data_2_univ_df.xlsx", index=False, header=True)

In [38]:
## Do some checking of the data by counting project names
## How many unique projects (not including subawards) were done in all 3 years by Northwestern University Chicago?

## From reporter_data_2_agg_df

## Filter to include ONLY projects with university of interest 

univ = ['NORTHWESTERN UNIVERSITY AT CHICAGO'] 
    
## selecting rows based on condition 
test_agg_df = reporter_data_2_agg_df.loc[reporter_data_2_agg_df['organization_name'].isin(univ)]
# test_agg_df.head()

## counting unique_id (i.e. projects) across all years
n = len(pd.unique(test_agg_df['unique_id']))
print("Final unique project from agg data: ", n)

## From reporter_data_2_univ_df
test_univ_df = reporter_data_2_univ_df.loc[reporter_data_2_univ_df['organization_name'].isin(univ)]
# test_univ_df.head()

## Get unique_id_list for each year

unique_id_2019 = test_univ_df.loc[test_univ_df['fiscal_year'] == 2019, 'unique_id']
# print(unique_id_2019)
unique_id_2020 = test_univ_df.loc[test_univ_df['fiscal_year'] == 2020, 'unique_id']
unique_id_2021 = test_univ_df.loc[test_univ_df['fiscal_year'] == 2021, 'unique_id']

unique_id_list_2019 = unique_id_2019.tolist()
# print(unique_id_list_2019[0])
# print(len(unique_id_list_2019[0]))
unique_id_list_2020 = unique_id_2020.tolist()
# print(len(unique_id_list_2020[0]))
unique_id_list_2021 = unique_id_2021.tolist() 
# print(len(unique_id_list_2021[0]))

unique_id_lists_all= unique_id_list_2019[0] + unique_id_list_2020[0] + unique_id_list_2021[0]
# print(unique_id_lists_all)


## Remove Dulicates
# final_list = list(set(unique_id_lists_all_2))
final_list = [*set(unique_id_lists_all)]
print("Final unique projects from univ data: ", len(final_list))


Final unique project from agg data:  55
Final unique projects from univ data:  55


In [39]:
## Do some checking of the data by sum of total funding
## How many unique projects (not including subawards) were done in all 3 years by Northwestern University Chicago?

## Filter to include ONLY projects with university of interest 

univ = ['NORTHWESTERN UNIVERSITY AT CHICAGO'] 
    
## selecting rows based on condition 
test_sum_df = reporter_data_2_agg_df.loc[reporter_data_2_agg_df['organization_name'].isin(univ)]
test_sum_df.head()

## counting unique values
# m = len(pd.unique(test_agg_df['total_cost']))
# print("Final unique project from agg data: ", m)

,nih_spending_categorization,project_terms,project_title,public_health_relevance,administering_ic,application_id,award_notice_date,foa,project_number,type,...,total_cost_sub_projects,funding_ics,direct_cost_ic,indirect_cost_ic,nih_covid-19_response,total_cost_ic,link_reporter,unique_id,Region,total_cost_main_or_sub_projects
58,"Adolescent Sexual Activity; Alcoholism, Alcoho...",AIDS prevention;AIDS/HIV problem;Accounting;Ac...,Efficacy of Couples-Based HIV Prevention in Vu...,Project Narrative\nMen who have sex with men (...,NIAAA,9766168,8/6/2019,PA-14-127,5R01AA024065-04,5.0,...,NaN,[NIAAA],347960.0,160562.0,None,[508522.0],https://reporter.nih.gov/project-details/9766168,AA024065,Midwest,508522.0
59,"Adolescent Sexual Activity; Alcoholism, Alcoho...",AIDS prevention;AIDS/HIV problem;Accounting;Ac...,Efficacy of Couples-Based HIV Prevention in Vu...,Project Narrative\nMen who have sex with men (...,NIAAA,9989734,8/29/2020,PA-14-127,5R01AA024065-05,5.0,...,NaN,[NIAAA],300237.0,142809.0,None,[443046.0],https://reporter.nih.gov/project-details/9989734,AA024065,Midwest,443046.0
61,"Adolescent Sexual Activity; Alcoholism, Alcoho...",Address;Adolescent Risk Behavior;Alcohol abuse...,Role of Alcohol Disparities in HIV Risk among ...,PUBLIC HEALTH RELEVANCE: We propose to utilize...,NIAAA,9648067,2/20/2019,PAR-14-338,5R01AA024409-04,5.0,...,NaN,[NIAAA],221761.0,92867.0,None,[314628.0],https://reporter.nih.gov/project-details/9648067,AA024409,Midwest,314628.0
62,"Adolescent Sexual Activity; Alcoholism, Alcoho...",Address;Adolescent Risk Behavior;Alcohol abuse...,Role of Alcohol Disparities in HIV Risk among ...,PUBLIC HEALTH RELEVANCE: We propose to utilize...,NIAAA,9878033,2/24/2020,PAR-14-338,5R01AA024409-05,5.0,...,NaN,[NIAAA],244839.0,104605.0,None,[349444.0],https://reporter.nih.gov/project-details/9878033,AA024409,Midwest,349444.0
180,"Adolescent Sexual Activity; Alcoholism, Alcoho...",AIDS prevention;Address;Adherence;Alcohol cons...,Investigating the Influence of Alcohol Use and...,Project Narrative:\nThe primary goal of the pr...,NIAAA,9926553,9/13/2019,PA-19-188,1F32AA028194-01,1.0,...,NaN,[NIAAA],61226.0,0.0,None,[61226.0],https://reporter.nih.gov/project-details/9926553,AA028194,Midwest,61226.0


In [40]:
print(test_sum_df['serial_number'])

58      024065
59      024065
61      024409
62      024409
180     028194
         ...  
8171    010350
8212    010946
8213    010946
8214    010946
8316    012092
Name: serial_number, Length: 113, dtype: object


In [41]:
## Export file to excel, without the Pandas index, but with the header

test_sum_df.to_excel("output/test_sum_df.xlsx", index=False, header=True)

PermissionError: [Errno 13] Permission denied: 'output/test_sum_df.xlsx'